# Imports packages and assign variables

In [ ]:
import pandas as pd
import os
from datetime import timedelta
import re
import csv
file_paths = [
    'testData/21.10_A+NA+AZA_30s_600ms_3/',
    'testData/22.130_A+NA+AZA_30s_600ms_3/',
    'testData/23.60_A+NA+AZA_30s_200ms_3/',
    'testData/24.60_A+NA+AZA_30s_1100ms_3/',
    'testData/25.60_A+NA+AZA_5s_600ms_3/',
    'testData/26.60_A+NA+AZA_50s_600ms_3/',
    'testData/27.60_A+NA+AZA_30s_600ms_2/',
    'testData/28.60_A+NA+AZA_30s_600ms_8/',
]

labels = [ _[12:-1] if _ [11] == '.' else _[11:-1] for _ in file_paths ]

# Convert .sql to .csv

In [ ]:
def to_snake_case(column_name):
    column_name = column_name.replace('"', '')
    column_name  = re.sub(r'(?<!^)(?=[A-Z])', '_', column_name).lower()
    if column_name == 'requestid':
        return 'request_id'
    return  column_name

def extract_table_data(sql_file, output_folder):
    with open(sql_file, 'r', encoding='utf-8') as file:
        content = file.read()
    pattern = r'COPY\s+public\."(?P<table_name>\w+)"\s+\((?P<columns>[^\)]+)\)\s+FROM\s+stdin;\n(?P<data>.*?)\\\.\n'
    matches = re.finditer(pattern, content, re.DOTALL)

    for match in matches:
        table_name = match.group('table_name')
        columns = match.group('columns').split(', ')
        data = match.group('data').strip()
        columns = [to_snake_case(column) for column in columns]
        rows = [row.split('\t') for row in data.splitlines()]
        csv_file = os.path.join(output_folder, f'{table_name}.csv')

        with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(columns)
            writer.writerows(rows)

        print(f'Data from table "{table_name}" has been written to "{csv_file}".')
test_params = [
    
    "10_A+NA+AZA_30s_600ms_3", "130_A+NA+AZA_30s_600ms_3","60_A+NA+AZA_30s_200ms_3", "60_A+NA+AZA_30s_1100ms_3", 
    "60_A+NA+AZA_5s_600ms_3", "60_A+NA+AZA_50s_600ms_3", "60_A+NA+AZA_30s_600ms_2", "60_A+NA+AZA_30s_600ms_8"
]
base_output_folder = 'testData'
if not os.path.exists(base_output_folder):
    os.makedirs(base_output_folder)
i = 20
for param in test_params:
    sql_file = f'logdatabase_test{i+1}.sql'
    numbered_folder_name = f"{i+1}.{param}"
    output_folder = os.path.join(base_output_folder, numbered_folder_name)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    print(f'Processing "{sql_file}" into folder "{output_folder}"...')
    extract_table_data(os.path.join('testDataSql/', sql_file), output_folder)
    i += 1
print("All SQL files have been processed.")


# Sort data by timestamp

In [ ]:
def sort():
    current_dir = os.getcwd()
    
    for test_dir in file_paths:
        test_dir_path = os.path.join(current_dir, test_dir)
        
        for root, _, files in os.walk(test_dir_path):
            for file in files:
                if file.endswith('.csv'):
                    file_path = os.path.join(root, file)
                    df = pd.read_csv(file_path)
                    df = df.sort_values(by='timestamp')
                    df.to_csv(file_path, index=False)
                    print(f'Sorted and saved: {file_path}')

sort()

# Get complete market logs

In [ ]:
def get_complete_market_logs():
    current_dir = os.getcwd()
    for test_dir in file_paths:
        test_dir_path = os.path.join(current_dir, test_dir)
        for root, _, files in os.walk(test_dir_path):
            if 'stockApp_trafficlog.csv' in files and 'stockApp_marketlog.csv' in files:
                traffic_log_path = os.path.join(root, test_dir_path,'stockApp_trafficlog.csv')
                market_log_path = os.path.join(root, test_dir_path,'stockApp_marketlog.csv')
                traffic_log_df = pd.read_csv(traffic_log_path)
                market_log_df = pd.read_csv(market_log_path)
                if 'user_class' in traffic_log_df.columns:
                    complete_log_df = pd.merge(market_log_df, traffic_log_df[['request_id', 'api_time','user_class']], on='request_id', how='inner')
                else:
                    complete_log_df = pd.merge(market_log_df, traffic_log_df[['request_id', 'api_time']], on='request_id', how='inner')
                if 'user_class' in complete_log_df.columns:
                    complete_log_df = complete_log_df[['id', 'timestamp', 'api_method', 'application_time', 'database_time', 'endpoint_url', 'api_time','user_class']]
                else:
                    complete_log_df = complete_log_df[['id', 'timestamp', 'api_method', 'application_time', 'database_time', 'endpoint_url', 'api_time']]
                complete_log_df = complete_log_df[~complete_log_df['endpoint_url'].isin(['/api/signIn', '/api/signUp', '/api/addCompany'])]
                complete_log_df['timestamp'] = pd.to_datetime(complete_log_df['timestamp'], errors='coerce').dt.tz_localize(None)
                complete_log_df = complete_log_df.dropna(subset=['timestamp'])
                output_path = os.path.join(root, 'stockApp_complete_marketlog.csv')
                complete_log_df.to_csv(output_path, index=False)
                print(f"File saved in: {output_path}")

get_complete_market_logs()

# Get transactions times

In [ ]:
def find_closest_timestamp(df, target_timestamp):
    df['timestamp'] = df['timestamp'].dt.tz_localize(None)
    if isinstance(target_timestamp, pd.Timestamp):
        target_timestamp = target_timestamp.tz_localize(None) if target_timestamp.tzinfo else target_timestamp
    return df.iloc[(df['timestamp'] - target_timestamp).abs().argsort()[:1]]['timestamp'].values[0]

def extract_companies_group_trade():
    current_dir = os.getcwd()
    for test_dir in file_paths:
        test_dir_path = os.path.join(current_dir, test_dir)
        for root, _, files in os.walk(test_dir_path):
            for file in files:
                if not file.endswith('.csv'):
                    continue
                if file != 'stockApp_tradelog.csv':
                    continue
                df = pd.read_csv(os.path.join(root, test_dir_path, 'stockApp_tradelog.csv'))
                df['company_ids'] = df['company_ids'].apply(lambda x: str(sorted(eval(x))))
                df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce').dt.tz_localize(None)
                df = df[df['timestamp'] <= df['timestamp'].min() + timedelta(hours=1) + timedelta(seconds=4)]
                unique_company_ids = df['company_ids'].unique()
                dfs = []
                for company_id_str in unique_company_ids:
                    company_df = df[df['company_ids'] == company_id_str].drop(columns=['company_ids']).reset_index(drop=True)
                    company_df['timestamp'] = pd.to_datetime(company_df['timestamp'], errors='coerce')
                    dfs.append(company_df)
                common_timestamps = sorted(set(dfs[0]['timestamp'].to_list()))
                for df_group in dfs[1:]:
                    for ts in common_timestamps:
                        if ts not in df_group['timestamp'].values:
                            closest_ts = find_closest_timestamp(df_group, ts)
                            common_timestamps.remove(ts)
                            common_timestamps.append(closest_ts)
                common_timestamps = [
                    pd.to_datetime(ts).tz_localize(None) if isinstance(ts, pd.Timestamp) else pd.to_datetime(ts)
                    for ts in common_timestamps
                ]
                result_df = pd.DataFrame([{
                        'timestamp': min(common_timestamps) - timedelta(minutes=2),
                        'application_time_trade': 0,
                        'database_time_trade': 0,
                        'number_of_sell_offers': 0,
                        'number_of_buy_offers': 0
                    }])
                for ts in common_timestamps:
                    pre_row = {
                        'timestamp': ts - timedelta(seconds=0.1),
                        'application_time_trade': 0,
                        'database_time_trade': 0,
                        'number_of_sell_offers': 0,
                        'number_of_buy_offers': 0
                    }
                    current_row = {
                        'timestamp': ts,
                        'application_time_trade': sum([df_group[df_group['timestamp'] == find_closest_timestamp(df_group, ts)]['application_time'].values[0] for df_group in dfs]) / len(dfs),
                        'database_time_trade': sum([df_group[df_group['timestamp'] == find_closest_timestamp(df_group, ts)]['database_time'].values[0] for df_group in dfs]) / len(dfs),
                        'number_of_sell_offers': sum([df_group[df_group['timestamp'] == find_closest_timestamp(df_group, ts)]['number_of_sell_offers'].values[0] for df_group in dfs]),
                        'number_of_buy_offers': sum([df_group[df_group['timestamp'] == find_closest_timestamp(df_group, ts)]['number_of_buy_offers'].values[0] for df_group in dfs])
                    }
                    max_application_time = max(df_group[df_group['timestamp'] == find_closest_timestamp(df_group, ts)]['application_time'].values[0] for df_group in dfs)
                    post_row = {
                        'timestamp': ts + timedelta(seconds=0.1 + 1.5 * max_application_time),
                        'application_time_trade': 0,
                        'database_time_trade': 0,
                        'number_of_sell_offers': 0,
                        'number_of_buy_offers': 0
                    }
                    result_df = pd.concat([result_df, pd.DataFrame([pre_row]), pd.DataFrame([current_row]), pd.DataFrame([post_row])], ignore_index=True)
                output_path = os.path.join(root, "stockApp_tradelog_mean.csv")
                result_df.to_csv(output_path, index=False)
                print(f"File saved in: {output_path}")

extract_companies_group_trade()

# Separate containers into files

In [ ]:
def extract_containers_cpu_usage():
    current_dir = os.getcwd()
    for test_dir in file_paths:
        test_dir_path = os.path.join(current_dir, test_dir)
        for root, _, files in os.walk(test_dir_path):
            for file in files:
                if not file.endswith('.csv'):
                    continue
                if file != 'stockApp_cpu.csv':
                    continue
                df = pd.read_csv(os.path.join(root, test_dir_path,'stockApp_cpu.csv'))
                contener_ids = ['stockproject-web-1',
                                'stockproject-db-1', 
                                'stockproject-db_test-1',
                                'stockproject-locust-1',
                                'stockproject-celery_worker_execute_transactions-1']
                contener_names =['web','db','db_test','test','transaction']
                for i,contener_id_str in  enumerate(contener_ids,start=0):
                    contener_df = df[df['contener_id'] == contener_id_str].drop(columns=['memory_usage','contener_id'])
                    contener_df['timestamp'] = pd.to_datetime(contener_df['timestamp'], errors='coerce').dt.tz_localize(None)
                    contener_df['timestamp'] = contener_df['timestamp'] - timedelta(hours=2)
                    contener_df.rename(columns={'cpu_usage': f'cpu_usage_{contener_names[i]}'},inplace=True)
                    file_name = f"stockApp_cpu_{contener_names[i]}.csv"
                    output_path = os.path.join(root,file_name)
                    contener_df.to_csv(output_path, index=False)
                    print(f"File saved in: {output_path}")
extract_containers_cpu_usage()

# Merge data for analysis

In [ ]:
def mergeData():
    for test_dir in file_paths:
        dfs = []
        dfs.append(pd.read_csv(f'{test_dir}/stockApp_complete_marketlog.csv'))
        dfs.append(pd.read_csv(f'{test_dir}/stockApp_tradelog_mean.csv'))
        dfs.append(pd.read_csv(f'{test_dir}/stockApp_cpu_db_test.csv'))
        dfs.append(pd.read_csv(f'{test_dir}/stockApp_cpu_db.csv'))
        dfs.append(pd.read_csv(f'{test_dir}/stockApp_cpu_test.csv'))
        dfs.append(pd.read_csv(f'{test_dir}/stockApp_cpu_transaction.csv'))
        dfs.append(pd.read_csv(f'{test_dir}/stockApp_cpu_web.csv'))
        for df in dfs:
            df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
            nan_rows = df[df['timestamp'].isna()]
            if(len(nan_rows.index) >=1):
                print(nan_rows)
                df = df.drop(df[df['timestamp'].isna()])
            df = df.drop(columns=[col for col in df.columns if col.startswith('id')], errors='ignore')
        merged_data = pd.merge_asof(dfs[0], 
                              dfs[1].sort_values('timestamp'), 
                              on='timestamp', 
                              direction='nearest')
        for i in range(2,len(dfs),1):
            merged_data = pd.merge_asof(merged_data, 
                              dfs[i], 
                              on='timestamp', 
                              direction='nearest')
            merged_data = merged_data.drop(columns=[col for col in merged_data.columns if col.startswith('id')], errors='ignore')
        
        merged_data.to_csv(f'{test_dir}/stockApp_merged_data.csv', index=False)
        if os.path.exists(f'{test_dir}/merged_data.csv'):
            os.remove(f'{test_dir}/merged_data.csv')
        print(f"File saved in: {test_dir}stockApp_merged_data.csv'")

mergeData()

In [ ]:
import pathlib
import pandas as pd
import matplotlib.pyplot as plt

TEST_DATA_DIR = pathlib.Path('testData')

test_dirs = sorted([p for p in TEST_DATA_DIR.iterdir() if p.is_dir()])

print(f' {len(test_dirs)} folders')

for test_dir in test_dirs:
    label = test_dir.name
    merged_file = test_dir / 'stockApp_merged_data.csv'
    traffic_file = test_dir / 'stockApp_trafficlog.csv' 

    if merged_file.exists():
        df = pd.read_csv(merged_file, parse_dates=['timestamp'])
    elif traffic_file.exists():
        df = pd.read_csv(traffic_file, parse_dates=['timestamp'])
    else:
        print(f'Lack merged/traffic file in {label} ')
        continue

    total = len(df)
    by_class = df['user_class'].value_counts(dropna=False)
    by_endpoint = df['endpoint_url'].value_counts(dropna=False)

    # Requests‑per‑second
    df['timestamp_sec'] = df['timestamp'].dt.floor('s')
    rps = df.groupby('timestamp_sec').size()

    print(f'\n=== Test {label} ===')
    print(f'Requests: {total}')

    # 1. RPS
    plt.figure()
    rps.plot()
    plt.title(f'RPS – {label}')
    plt.xlabel('Time')
    plt.ylabel('Requests / s')
    plt.tight_layout()
    plt.show()

    # 2. Class
    plt.figure()
    by_class.plot(kind='bar')
    plt.title(f'user_class – {label}')
    plt.ylabel('Requests')
    plt.tight_layout()
    plt.show()

    # 3. Endpoint
    plt.figure()
    by_endpoint.head(20).plot(kind='bar')
    plt.title(f'Endpoints – {label}')
    plt.ylabel('Requests')
    plt.tight_layout()
    plt.show()
